# The Cancer Gene Atlas Cancer (TCGA) - Uterine Carcinosarcoma (UCS)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.5 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 6.3.1
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.14.2
pandas 0.22.0
clustergrammer_widget 1.9.0
seaborn 0.8.1
Fri May 11 16:16:51 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [10]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [11]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [12]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [13]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Uterine Carcinosarcoma [UCS]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-N5-A4RV,505,UCS,TCGA,3AF5B391-E72F-463D-A086-A86C6C30A51A,N5,62.0,Unknown - Uterus NOS,302,Nationwide Children's Hospital,NaN,...,23.8,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-N5-A4RS,506,UCS,TCGA,4DAEEFC3-0E2E-4537-8E13-AF9717B3CE29,N5,65.0,Unknown - Uterus NOS,302,Nationwide Children's Hospital,NaN,...,25.0,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-N5-A4RO,507,UCS,TCGA,4F4906DC-7EBD-47F1-A8F5-B35D3950E740,N5,67.0,Unknown - Uterus NOS,302,Nationwide Children's Hospital,NaN,...,28.3,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-N5-A4RD,508,UCS,TCGA,D9952E44-B573-4854-893E-B3BBC5287F2F,N5,69.0,Unknown - Uterus NOS,302,Nationwide Children's Hospital,NaN,...,23.3,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-N5-A4R8,509,UCS,TCGA,8C113BC1-101B-405F-9EB1-8C9E8EBBE185,N5,65.0,Unknown - Uterus NOS,302,Nationwide Children's Hospital,NaN,...,20.1,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
sample_meta.shape

(57, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_UCS_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,6b86e544-0372-434a-9f28-6fa4075dd228,f4866b8f-4e58-4c17-8fe1-0190f57f3d13,8e5368d5-de3c-4e98-baac-61a3694e1a60,2122e194-ba31-42bd-bc2c-4aa0ec65ef5f,c95f3a7f-768c-4728-a8be-22a67f3543a6,6cf8dd8a-69b8-488c-ae0e-cdbd6c14b604,5c266b88-cabe-4b19-806d-97369b64ddb2,c501be79-8adc-4e90-8b1d-a1d8bb575443,02620968-bac9-47b8-b21a-1c554224137a,68d35de3-71c6-4adc-8699-a750aeed46f5,...,34631edd-0e4c-43a0-84cb-40bb61b05df6,239d877f-18e4-4a67-b995-780f646f1b45,d72f5821-70ea-4f0d-8a61-16ace356a19a,02a39665-8dca-4723-9397-b3ed78293717,4887d9a5-2ea9-4e37-bb4a-5b12593a23b3,8349b5f6-8fba-47fc-b0d3-d4461aaa4719,9d9c9c00-fd4b-47c2-9ece-bc359c281e58,efb64e97-ef61-4bee-95cd-13d8fc5062f7,6742c0f8-19f0-42db-82db-bdd0258226e9,a84b3525-fc44-4920-bcda-fb9a87915cad
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,255.842000,136.151000,118.89300,238.75100,167.542000,221.226000,162.87700,322.50800,406.18100,93.924700,...,241.65400,85.526100,224.05000,121.2360,124.19200,252.91300,182.67700,212.6200,588.06000,279.8170
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,2.925610,47.720100,9.20521,46.56180,10.699900,10.376700,29.95070,6.55237,6.13674,45.350200,...,32.67170,9.727520,12.22330,19.1845,16.34510,17.27120,25.52030,26.1338,1.66236,14.3170
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,9.078050,7.081430,28.13740,9.50002,9.860090,9.356840,9.78689,4.22144,16.87870,23.647400,...,18.72470,10.454100,17.22320,24.5537,26.08760,20.91720,3.20500,16.5344,8.92621,10.7147
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,93.567100,224.757000,101.97500,115.45300,60.486200,119.021000,131.16800,27.69460,94.64530,132.211000,...,92.45050,64.438200,73.41560,41.4933,70.74010,40.02620,53.38160,98.6186,129.24400,98.6849
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.896853,0.146054,0.00000,1.99596,0.116025,0.346664,4.97439,4.94919,0.00000,0.307593,...,1.10858,0.154698,1.59944,2.5177,3.34406,3.20327,2.63477,2.1079,0.00000,0.0000


In [19]:
matrix.shape

(199169, 57)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

,6b86e544-0372-434a-9f28-6fa4075dd228,f4866b8f-4e58-4c17-8fe1-0190f57f3d13,8e5368d5-de3c-4e98-baac-61a3694e1a60,2122e194-ba31-42bd-bc2c-4aa0ec65ef5f,c95f3a7f-768c-4728-a8be-22a67f3543a6,6cf8dd8a-69b8-488c-ae0e-cdbd6c14b604,5c266b88-cabe-4b19-806d-97369b64ddb2,c501be79-8adc-4e90-8b1d-a1d8bb575443,02620968-bac9-47b8-b21a-1c554224137a,68d35de3-71c6-4adc-8699-a750aeed46f5,...,34631edd-0e4c-43a0-84cb-40bb61b05df6,239d877f-18e4-4a67-b995-780f646f1b45,d72f5821-70ea-4f0d-8a61-16ace356a19a,02a39665-8dca-4723-9397-b3ed78293717,4887d9a5-2ea9-4e37-bb4a-5b12593a23b3,8349b5f6-8fba-47fc-b0d3-d4461aaa4719,9d9c9c00-fd4b-47c2-9ece-bc359c281e58,efb64e97-ef61-4bee-95cd-13d8fc5062f7,6742c0f8-19f0-42db-82db-bdd0258226e9,a84b3525-fc44-4920-bcda-fb9a87915cad
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,255.842000,136.151000,118.89300,238.75100,167.542000,221.226000,162.87700,322.50800,406.18100,93.924700,...,241.65400,85.526100,224.05000,121.2360,124.19200,252.91300,182.67700,212.6200,588.06000,279.8170
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,2.925610,47.720100,9.20521,46.56180,10.699900,10.376700,29.95070,6.55237,6.13674,45.350200,...,32.67170,9.727520,12.22330,19.1845,16.34510,17.27120,25.52030,26.1338,1.66236,14.3170
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,9.078050,7.081430,28.13740,9.50002,9.860090,9.356840,9.78689,4.22144,16.87870,23.647400,...,18.72470,10.454100,17.22320,24.5537,26.08760,20.91720,3.20500,16.5344,8.92621,10.7147
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,93.567100,224.757000,101.97500,115.45300,60.486200,119.021000,131.16800,27.69460,94.64530,132.211000,...,92.45050,64.438200,73.41560,41.4933,70.74010,40.02620,53.38160,98.6186,129.24400,98.6849
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.896853,0.146054,0.00000,1.99596,0.116025,0.346664,4.97439,4.94919,0.00000,0.307593,...,1.10858,0.154698,1.59944,2.5177,3.34406,3.20327,2.63477,2.1079,0.00000,0.0000


In [22]:
matrix.shape

(79930, 57)

# Map Sample Meta Data to Sample ID

In [23]:
getSampleID(matrix, meta)

In [24]:
matrix.head()

,TCGA-N5-A4RV-01,TCGA-N6-A4VC-01,TCGA-N7-A4Y0-01,TCGA-N6-A4V9-01,TCGA-N5-A59F-01,TCGA-N5-A59E-01,TCGA-N5-A4RO-01,TCGA-N5-A4RN-01,TCGA-N5-A4RS-01,TCGA-N5-A4R8-01,...,TCGA-NF-A4X2-01,TCGA-ND-A4W6-01,TCGA-QN-A5NN-01,TCGA-NA-A4QW-01,TCGA-NA-A5I1-01,TCGA-ND-A4WF-01,TCGA-ND-A4WA-01,TCGA-NG-A4VW-01,TCGA-QM-A5NM-01,TCGA-N5-A4RM-01
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,255.842000,136.151000,118.89300,238.75100,167.542000,221.226000,162.87700,322.50800,406.18100,93.924700,...,241.65400,85.526100,224.05000,121.2360,124.19200,252.91300,182.67700,212.6200,588.06000,279.8170
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,2.925610,47.720100,9.20521,46.56180,10.699900,10.376700,29.95070,6.55237,6.13674,45.350200,...,32.67170,9.727520,12.22330,19.1845,16.34510,17.27120,25.52030,26.1338,1.66236,14.3170
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,9.078050,7.081430,28.13740,9.50002,9.860090,9.356840,9.78689,4.22144,16.87870,23.647400,...,18.72470,10.454100,17.22320,24.5537,26.08760,20.91720,3.20500,16.5344,8.92621,10.7147
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,93.567100,224.757000,101.97500,115.45300,60.486200,119.021000,131.16800,27.69460,94.64530,132.211000,...,92.45050,64.438200,73.41560,41.4933,70.74010,40.02620,53.38160,98.6186,129.24400,98.6849
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.896853,0.146054,0.00000,1.99596,0.116025,0.346664,4.97439,4.94919,0.00000,0.307593,...,1.10858,0.154698,1.59944,2.5177,3.34406,3.20327,2.63477,2.1079,0.00000,0.0000


# Map Gene Symbol to Gene ID

In [25]:
getGenes(matrix)

In [26]:
matrix.head()

,TCGA-N5-A4RV-01,TCGA-N6-A4VC-01,TCGA-N7-A4Y0-01,TCGA-N6-A4V9-01,TCGA-N5-A59F-01,TCGA-N5-A59E-01,TCGA-N5-A4RO-01,TCGA-N5-A4RN-01,TCGA-N5-A4RS-01,TCGA-N5-A4R8-01,...,TCGA-NF-A4X2-01,TCGA-ND-A4W6-01,TCGA-QN-A5NN-01,TCGA-NA-A4QW-01,TCGA-NA-A5I1-01,TCGA-ND-A4WF-01,TCGA-ND-A4WA-01,TCGA-NG-A4VW-01,TCGA-QM-A5NM-01,TCGA-N5-A4RM-01
ARF5,255.842000,136.151000,118.89300,238.75100,167.542000,221.226000,162.87700,322.50800,406.18100,93.924700,...,241.65400,85.526100,224.05000,121.2360,124.19200,252.91300,182.67700,212.6200,588.06000,279.8170
M6PR,2.925610,47.720100,9.20521,46.56180,10.699900,10.376700,29.95070,6.55237,6.13674,45.350200,...,32.67170,9.727520,12.22330,19.1845,16.34510,17.27120,25.52030,26.1338,1.66236,14.3170
ESRRA,9.078050,7.081430,28.13740,9.50002,9.860090,9.356840,9.78689,4.22144,16.87870,23.647400,...,18.72470,10.454100,17.22320,24.5537,26.08760,20.91720,3.20500,16.5344,8.92621,10.7147
FKBP4,93.567100,224.757000,101.97500,115.45300,60.486200,119.021000,131.16800,27.69460,94.64530,132.211000,...,92.45050,64.438200,73.41560,41.4933,70.74010,40.02620,53.38160,98.6186,129.24400,98.6849
CYP26B1,0.896853,0.146054,0.00000,1.99596,0.116025,0.346664,4.97439,4.94919,0.00000,0.307593,...,1.10858,0.154698,1.59944,2.5177,3.34406,3.20327,2.63477,2.1079,0.00000,0.0000


# Histogram of First Sample

In [27]:
sns.distplot(matrix.iloc[:, 0]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [28]:
sns.distplot(matrix.iloc[0, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Unfiltered Matrix to File

In [29]:
filename = path+'tcga_ucs_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [30]:
normalized_matrix = matrix.copy()

In [31]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining missing values to matrix mean

In [32]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [33]:
normalized_matrix.head()

,TCGA-N5-A4RV-01,TCGA-N6-A4VC-01,TCGA-N7-A4Y0-01,TCGA-N6-A4V9-01,TCGA-N5-A59F-01,TCGA-N5-A59E-01,TCGA-N5-A4RO-01,TCGA-N5-A4RN-01,TCGA-N5-A4RS-01,TCGA-N5-A4R8-01,...,TCGA-NF-A4X2-01,TCGA-ND-A4W6-01,TCGA-QN-A5NN-01,TCGA-NA-A4QW-01,TCGA-NA-A5I1-01,TCGA-ND-A4WF-01,TCGA-ND-A4WA-01,TCGA-NG-A4VW-01,TCGA-QM-A5NM-01,TCGA-N5-A4RM-01
A1BG,0.027062,0.067056,0.038446,0.067056,0.067056,0.067056,0.067056,0.067056,0.067056,0.067056,...,0.069047,0.067056,0.067056,0.067056,0.061887,0.067056,0.067056,0.067056,0.067056,0.067056
A1CF,0.004713,0.003116,0.158485,0.002671,0.016866,1.561780,0.017179,0.011469,0.040150,0.018430,...,0.015220,0.013771,0.158485,0.024428,0.198162,0.002756,0.014606,0.009926,0.005225,0.003782
A2M,321.498000,335.554000,14.945700,104.629000,35.268500,82.251300,98.829600,124.379000,34.526100,38.372200,...,72.241500,134.309000,141.815000,140.636000,131.418000,309.102000,65.525900,70.177000,14.985400,34.062000
A2ML1,0.075573,0.105829,0.026510,0.781507,0.094270,0.277441,0.125762,0.339650,0.270449,0.418480,...,0.075028,0.101461,1.223620,0.056733,1.207610,0.027202,0.046824,1.275020,1.580990,0.016016
A3GALT2,0.042305,0.049224,0.056863,0.044979,0.056863,0.056863,0.020530,0.038982,0.049975,0.175822,...,0.022447,0.056863,0.056863,0.022986,0.056863,0.023166,0.056863,0.056863,0.104114,0.056863


In [34]:
normalized_matrix.shape

(19363, 57)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [35]:
samples = normalized_matrix.columns.tolist()

In [36]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [37]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [38]:
normalized_matrix.columns = samples

In [39]:
normalized_matrix.head()

,TCGA-N5-A4RV-01,TCGA-N6-A4VC-01,TCGA-N7-A4Y0-01,TCGA-N6-A4V9-01,TCGA-N5-A59F-01,TCGA-N5-A59E-01,TCGA-N5-A4RO-01,TCGA-N5-A4RN-01,TCGA-N5-A4RS-01,TCGA-N5-A4R8-01,...,TCGA-NF-A4X2-01,TCGA-ND-A4W6-01,TCGA-QN-A5NN-01,TCGA-NA-A4QW-01,TCGA-NA-A5I1-01,TCGA-ND-A4WF-01,TCGA-ND-A4WA-01,TCGA-NG-A4VW-01,TCGA-QM-A5NM-01,TCGA-N5-A4RM-01
A1BG,-1.361935,-0.978600,-1.213483,-0.977526,-0.960460,-0.946601,-0.966255,-0.966841,-0.931313,-0.963288,...,-0.986113,-0.944708,-0.982126,-0.940784,-0.970176,-0.958546,-0.967718,-0.977169,-0.947006,-0.964139
A1CF,-2.121030,-2.311485,-0.598345,-2.377255,-1.559899,0.420578,-1.557690,-1.733756,-1.154068,-1.524206,...,-1.642839,-1.632196,-0.608580,-1.379345,-0.464756,-2.344760,-1.629615,-1.806824,-2.055364,-2.212893
A2M,2.712890,2.720722,1.376184,2.215686,1.760487,2.142101,2.202192,2.301465,1.780395,1.794288,...,2.033530,2.356957,2.343156,2.380872,2.356879,2.705115,2.022255,2.042585,1.402222,1.741691
A2ML1,-0.915922,-0.780436,-1.374919,0.088967,-0.812528,-0.329871,-0.693146,-0.262250,-0.325668,-0.168054,...,-0.950032,-0.764849,0.279080,-1.013389,0.320151,-1.350392,-1.123685,0.301907,0.425483,-1.586025
A3GALT2,-1.167896,-1.112863,-1.043504,-1.150956,-1.032072,-1.018213,-1.480311,-1.202423,-1.058998,-0.544655,...,-1.474089,-1.016320,-1.053738,-1.405765,-1.006948,-1.420141,-1.039330,-1.048782,-0.755937,-1.035751


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [40]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [41]:
normalized_matrix.head()

,TCGA-N5-A4RV-01,TCGA-N6-A4VC-01,TCGA-N7-A4Y0-01,TCGA-N6-A4V9-01,TCGA-N5-A59F-01,TCGA-N5-A59E-01,TCGA-N5-A4RO-01,TCGA-N5-A4RN-01,TCGA-N5-A4RS-01,TCGA-N5-A4R8-01,...,TCGA-NF-A4X2-01,TCGA-ND-A4W6-01,TCGA-QN-A5NN-01,TCGA-NA-A4QW-01,TCGA-NA-A5I1-01,TCGA-ND-A4WF-01,TCGA-ND-A4WA-01,TCGA-NG-A4VW-01,TCGA-QM-A5NM-01,TCGA-N5-A4RM-01
A1BG,-1.274430,-1.010838,-1.032938,-0.986918,-0.990734,-1.045725,-0.987528,-0.925559,-1.059160,-0.983435,...,-0.821654,-0.911322,-0.897677,-1.052096,-1.132069,-0.967417,-0.951407,-0.985085,-0.899862,-1.019626
A1CF,-2.094642,-2.380276,-0.290806,-2.737922,-1.574479,0.348911,-1.594957,-1.713921,-1.271285,-1.550844,...,-1.482479,-1.643232,-0.511126,-1.489270,-0.678805,-2.202702,-1.629081,-1.890276,-1.956775,-2.222555
A2M,2.656178,2.692661,1.484632,2.189823,1.798511,2.103362,2.249155,2.400900,1.743642,1.741894,...,1.987372,2.377985,2.307793,2.378903,2.337288,2.679507,1.988695,1.999568,1.495850,1.748725
A2ML1,-0.804147,-0.814567,-1.215704,0.036780,-0.855389,-0.466851,-0.712959,-0.199839,-0.464317,-0.191119,...,-0.781629,-0.725287,0.332993,-1.123895,0.114303,-1.388918,-1.114006,0.213591,0.617532,-1.692418
A3GALT2,-1.076373,-1.143029,-0.827213,-1.165508,-1.057918,-1.113718,-1.513114,-1.171848,-1.182369,-0.569515,...,-1.337710,-0.991423,-0.976022,-1.513685,-1.162867,-1.464764,-1.025011,-1.057918,-0.692841,-1.105294


# Normalize Matrix (z-score the rows)

In [42]:
uf.zscore(normalized_matrix, 'row')

In [43]:
normalized_matrix.head()

,TCGA-N5-A4RV-01,TCGA-N6-A4VC-01,TCGA-N7-A4Y0-01,TCGA-N6-A4V9-01,TCGA-N5-A59F-01,TCGA-N5-A59E-01,TCGA-N5-A4RO-01,TCGA-N5-A4RN-01,TCGA-N5-A4RS-01,TCGA-N5-A4R8-01,...,TCGA-NF-A4X2-01,TCGA-ND-A4W6-01,TCGA-QN-A5NN-01,TCGA-NA-A4QW-01,TCGA-NA-A5I1-01,TCGA-ND-A4WF-01,TCGA-ND-A4WA-01,TCGA-NG-A4VW-01,TCGA-QM-A5NM-01,TCGA-N5-A4RM-01
A1BG,-2.997068,-0.243507,-0.474362,0.006368,-0.033494,-0.607949,0.000000,0.647347,-0.748291,0.042758,...,1.732769,0.796068,0.938608,-0.674500,-1.509923,0.210082,0.377333,0.025524,0.915791,-0.335305
A1CF,-0.917743,-1.421698,2.264831,-2.052707,0.000000,3.393508,-0.036131,-0.246023,0.534936,0.041699,...,0.162318,-0.121303,1.876113,0.150336,1.580271,-1.108398,-0.096338,-0.557173,-0.674500,-1.143426
A2M,1.542842,1.656720,-2.113982,0.087178,-1.134251,-0.182699,0.272375,0.746025,-1.305516,-1.310973,...,-0.544746,0.674500,0.455406,0.677365,0.547471,1.615661,-0.540617,-0.506679,-2.078966,-1.289649
A2ML1,-0.247369,-0.275636,-1.363816,2.033845,-0.386376,0.667626,0.000000,1.391960,0.674500,1.415615,...,-0.186285,-0.033443,2.837395,-1.114763,2.244144,-1.833701,-1.087936,2.513488,3.609275,-2.657018
A3GALT2,0.000000,-0.470902,1.760228,-0.629708,0.130374,-0.263828,-3.085423,-0.674500,-0.748823,3.580775,...,-1.846253,0.600143,0.708944,-3.089454,-0.611048,-2.743849,0.362854,0.130374,2.709514,-0.204315


# Merge Like Column (by taking the mean)

In [44]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [45]:
normalized_matrix.shape

(19363, 57)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [46]:
uf.mapgenesymbols(normalized_matrix)

In [47]:
normalized_matrix.shape

(18913, 57)

# Merge Duplicate Genes By Rows

In [48]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [49]:
normalized_matrix.shape

(18821, 57)

# Histogram of First Sample

In [50]:
sns.distplot(normalized_matrix.iloc[:, 10]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [51]:
sns.distplot(normalized_matrix.iloc[10, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Filtered Matrix

In [52]:
filename = path+'tcga_ucs_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [53]:
gene_list = uf.createGeneList(normalized_matrix)

In [54]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [55]:
gene_list.shape

(18821, 2)

# Save Gene List

In [56]:
filename = path+'tcga_ucs_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [57]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [58]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [59]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-N5-A4R8-01,65,FEMALE,NaN,NaN,NaN,NaN,Uterine Carcinosarcoma/ Malignant Mixed Muller...,Primary Solid Tumor,Uterus,NaN
TCGA-N5-A4RA-01,63,FEMALE,NaN,NaN,NaN,NaN,Uterine Carcinosarcoma/ Malignant Mixed Muller...,Primary Solid Tumor,Uterus,NaN
TCGA-N5-A4RD-01,69,FEMALE,NaN,NaN,NaN,NaN,Uterine Carcinosarcoma/ Malignant Mixed Muller...,Primary Solid Tumor,Uterus,NaN
TCGA-N5-A4RF-01,68,FEMALE,NaN,NaN,NaN,NaN,Uterine Carcinosarcoma/ Malignant Mixed Muller...,Primary Solid Tumor,Uterus,NaN
TCGA-N5-A4RJ-01,61,FEMALE,NaN,NaN,NaN,NaN,Uterine Carcinosarcoma/ Malignant Mixed Muller...,Primary Solid Tumor,Uterus,NaN


In [60]:
attribute_list.shape

(57, 10)

# Save Attribute List

In [61]:
filename = path+'tcga_ucs_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [62]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [63]:
standard_matrix.head()

,TCGA-N5-A4R8-01,TCGA-N5-A4RA-01,TCGA-N5-A4RD-01,TCGA-N5-A4RF-01,TCGA-N5-A4RJ-01,TCGA-N5-A4RM-01,TCGA-N5-A4RN-01,TCGA-N5-A4RO-01,TCGA-N5-A4RS-01,TCGA-N5-A4RT-01,...,TCGA-ND-A4WC-01,TCGA-ND-A4WF-01,TCGA-NF-A4WU-01,TCGA-NF-A4WX-01,TCGA-NF-A4X2-01,TCGA-NF-A5CP-01,TCGA-NG-A4VU-01,TCGA-NG-A4VW-01,TCGA-QM-A5NM-01,TCGA-QN-A5NN-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0.128320,0.689977,0.971014,0.935895,0.233639,-0.327701,0.444294,-0.012046,-0.608302,-0.397875,...,0.584673,0.163385,-0.818702,-0.362862,0.830488,-0.152401,0.479262,0.058154,0.549555,0.619701
A1CF,0.012568,-0.899315,0.223132,-0.408292,0.855156,-0.794104,-0.303029,-0.092749,0.293292,-0.057611,...,0.574199,-0.759101,0.468867,0.819985,0.152906,0.082811,-0.583683,-0.478485,-0.548649,0.644335
A2M,-0.801377,-0.275287,-0.205184,0.742399,-0.590977,-0.731293,0.601998,0.180710,-0.766373,-0.871521,...,-0.170171,0.847813,-0.696235,0.145645,-0.380550,0.777541,0.672111,-0.310376,-0.906590,0.356167
A2ML1,0.643329,-0.444340,0.081782,-0.900287,0.222114,-0.935361,0.608301,0.011582,0.397553,-0.198881,...,-0.689932,-0.865218,0.187013,0.783844,-0.233943,0.573183,-0.795074,0.889219,0.994642,0.959523
A3GALT2,0.962358,-0.581872,-0.266195,0.470679,0.260026,-0.161019,-0.476639,-0.897501,-0.511779,0.189839,...,0.505716,-0.757287,0.821817,-0.090898,-0.616908,0.049528,-0.652055,0.119673,0.892037,0.540950


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [64]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [65]:
filename = path+'tcga_ucs_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [66]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [67]:
tertiary_matrix.head()

,TCGA-N5-A4R8-01,TCGA-N5-A4RA-01,TCGA-N5-A4RD-01,TCGA-N5-A4RF-01,TCGA-N5-A4RJ-01,TCGA-N5-A4RM-01,TCGA-N5-A4RN-01,TCGA-N5-A4RO-01,TCGA-N5-A4RS-01,TCGA-N5-A4RT-01,...,TCGA-ND-A4WC-01,TCGA-ND-A4WF-01,TCGA-NF-A4WU-01,TCGA-NF-A4WX-01,TCGA-NF-A4X2-01,TCGA-NF-A5CP-01,TCGA-NG-A4VU-01,TCGA-NG-A4VW-01,TCGA-QM-A5NM-01,TCGA-QN-A5NN-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
A3GALT2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Teriary Matrix

In [68]:
filename = path+'tcga_ucs_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [69]:
name = 'tcga_ucs_gene_up_set'

In [70]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [71]:
name = 'tcga_ucs_gene_down_set'

In [72]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [73]:
name = 'tcga_ucs_attribute_up_set'

In [74]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [75]:
name = 'tcga_ucs_attribute_down_set'

In [76]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity Matrix

In [77]:
attribute_normalized_matrix = normalized_matrix.copy()

In [78]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [79]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [80]:
attribute_similarity_matix.head()

,"(Donor:TCGA-N5-A4R8, Age:65.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-N5-A4RA, Age:63.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-N5-A4RD, Age:69.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-N5-A4RF, Age:68.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-N5-A4RJ, Age:61.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-N5-A4RM, Age:62.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-N5-A4RN, Age:71.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-N5-A4RO, Age:67.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-N5-A4RS, Age:65.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-N5-A4RT, Age:69.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)",...,"(Donor:TCGA-ND-A4WC, Age:83.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-ND-A4WF, Age:77.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-NF-A4WU, Age:59.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-NF-A4WX, Age:61.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-NF-A4X2, Age:59.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-NF-A5CP, Age:90.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-NG-A4VU, Age:63.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-NG-A4VW, Age:74.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-QM-A5NM, Age:67.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Uterine Carcinosarcoma/ Malignant Mixed Mullerian Tumor (MMMT): NOS, Tissue:Uterus, Tumor:nan, Gradenan)","(Donor:TCGA-QN-A5NN, Age:55.0, Gender:FEMALE, SampleType:Primary Solid Tumor, St

# Save Attribute Similarity Matrix

In [81]:
filename = path+'tcga_ucs_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [82]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

clustergrammer_widget(network='{"row_nodes": [{"cat_0_index": 17, "cat-6": "Tumor:nan", "cat-2": "SampleType:P…

# Create Gene Similarity Matrix

In [83]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [84]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,0.037339,-0.038814,-0.101652,-0.005687,0.213582,-0.142732,-0.286428,0.040019,0.239378,...,-0.088156,0.149802,-0.156580,-0.153230,-0.380979,-0.224203,-0.084345,0.097681,-0.098590,-0.079016
A1CF,0.037339,1.000000,-0.258016,0.020373,0.112371,-0.156176,0.033258,0.209937,-0.058581,-0.041984,...,0.149070,-0.015528,0.044751,0.108290,-0.054858,0.009776,0.013142,-0.069885,-0.112040,-0.086470
A2M,-0.038814,-0.258016,1.000000,-0.010290,-0.222382,0.137270,0.150287,0.086567,0.183053,-0.087697,...,-0.010647,0.011120,-0.141064,-0.263796,0.009489,-0.266131,0.054941,0.240655,-0.047065,0.359964
A2ML1,-0.101652,0.020373,-0.010290,1.000000,0.140060,-0.003546,0.150716,-0.135441,-0.207499,-0.049272,...,-0.100523,-0.158097,0.109754,0.108232,-0.034983,-0.037464,-0.153489,0.017702,-0.049102,-0.159235
A3GALT2,-0.005687,0.112371,-0.222382,0.140060,1.000000,0.074584,0.203025,0.032342,-0.131394,0.000686,...,-0.086735,0.106465,0.027114,-0.084243,-0.051692,0.152314,-0.032523,-0.065289,0.015687,-0.089940


# Save Gene Similarity Matrix

In [85]:
filename = path+'tcga_ucs_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [86]:
name = 'tcga_ucs_gene_attribute_edge_list'

In [87]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  57 Out of 57   

 The number of statisticaly relevent gene-attribute associations is: 56384
